## Assignments


In the following task, you'll continue working with the Credit Card Fraud Detection dataset from Kaggle. Before moving on to the tasks, load the dataset using Dask.

https://www.kaggle.com/mlg-ulb/creditcardfraud

To complete this assignment, submit your solutions to the following tasks as a link to your Jupyter Notebook on GitHub.

1. In this task, you'll train several machine-learning models from scikit-learn, using Dask as the backend of joblib. This time, you need to use all of the variables except Class as your feature set. The Class variable will be your target variable.

2. Compare the results of your models.


Submit your work below. You can also take a look at this example solution.

https://drive.google.com/file/d/14CliRImmsuIzBRSYV4Mi1CH2HDVTdumi/view?usp=sharing


In [1]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from dask_ml.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import roc_auc_score
import joblib
from dask.distributed import Client, progress
from dask_ml.model_selection import train_test_split
# DataFrames implement the Pandas API
import dask.dataframe as dd
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [6]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:52016 Dashboard: http://127.0.0.1:52015/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [7]:
# This loads the data into a Dask DataFrame
df = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/creditcard.csv', dtype={'Time': 'float64'})

In [8]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


1. In this task, you'll train several machine-learning models from scikit-learn, using Dask as the backend of joblib. This time, you need to use all of the variables except Class as your feature set. The Class variable will be your target variable.

In [9]:
# This is the feature set
X = df.drop("Class", axis=1)

# This is the target variable
Y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Because the data can fit into memory,
# persist it to the RAM
X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=3
    int64
      ...
      ...
      ...
Name: Class, dtype: int64
Dask Name: split, 3 tasks

In [16]:
lr = LogisticRegression()

with joblib.parallel_backend('dask'):
    lr.fit(X_train.values.compute(), y_train.values.compute())
    
preds_train = lr.predict(X_train.values.compute())
preds_test = lr.predict(X_test.values.compute())

print("Logistic regression training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Logistic regression test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Logistic regression training score is:  0.9410640624386895
Logistic regression test score is:  0.9503723186353689


In [18]:
gbc = GradientBoostingClassifier()

with joblib.parallel_backend('dask'):
    gbc.fit(X_train.values.compute(), y_train.values.compute())

preds_train = gbc.predict(X_train.values.compute())
preds_test = gbc.predict(X_test.values.compute())

print("Gradient boosting tree training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Gradient boosting tree test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Gradient boosting tree training score is:  0.8855301337354179
Gradient boosting tree test score is:  0.8652251120503348


In [19]:
rfc = RandomForestClassifier()

with joblib.parallel_backend('dask'):
    rfc.fit(X_train.values.compute(), y_train.values.compute())

preds_train = rfc.predict(X_train.values.compute())
preds_test = rfc.predict(X_test.values.compute())

print("Random forest training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Random forest test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Random forest training score is:  1.0
Random forest test score is:  0.987768346637998


2. Compare the results of your models.

Random Forest Classifier performs the best (both training and test), then the Logistic Regression and then the Gradient Boosting Classifier. 

We run these models using their default parameters. Ideally, we would/should do hyperparameter tuning by doing something like gridsearch or random search. These models took a bit of time to run!

In [20]:
client.close()